### Optimizador de Extracción de Sacarosa Tándem B

Se presenta el análisis y construcción del problema de optimización de *Extracción de Sacarosa* del Tándem B de Ingenio Pantaleon. El objetivo es determinar recomendaciones para variables controlables de operación del tándem de molinos que permitan simultáneamente:
- Maximizar la Extracción de Sacarosa.
- Operar dentro de los parámetros mecánicos de reductores.
- Operar dentro de los parámetros eléctricos de motores.
- Obtener una molienda horaria específica.
- Operar sin superar una humedad de bagazo máxima específica.


In [2]:
import numpy as np
import pandas as pd
from gurobipy import *


### Conjuntos y Notaciones

Se define la siguiente notación:

- $T_i$ el *Torque* (en kNm) aplicado en el Molino $i$, $\forall i \in \{1,2,3,4,5,6\}$

- $P_i$ la *Potencia* (en kW) aplicada al Molino $i$, $\forall i \in \{1,2,3,4,5,6\}$

- $S_i$ la *Velocidad* (en rpms) del motor eléctrico del Molino $i$, $\forall i \in \{1,2,3,4,5,6\}$

- $R_i$ el *Ratio de Reducción* del reductor del Molino $i$, $\forall i \in \{1,2,3,4,5,6\}$

Por tanto:

$$ T_i = \frac{60 * P_i}{2*\pi*\frac{S_i}{R_i}}$$

- $E$ el % de **Extracción de Sacarosa**.

- $F$ el % de Fibra en Caña.

- $B$ el % de Bagazo en Caña.

- $C$ el Rendimiento Core (kg/t) de la Caña.

- $D$ el día de Zafra.

- $W$ el Ratio de Molienda (t/h), en donde:
$$W = \beta_0 + \beta_1 * T_1 + \beta_2 * T_2 + \beta_3 * T_3 + \beta_4 * T_4 + \beta_5 * T_5 + \beta_6 * T_6 + \beta_7 * F$$

- $H$ el % de Humedad de Bagazo, en donde:
$$H = \gamma_0 + \gamma_1 * T_1 + \gamma_2 * T_6 + \gamma_3 * I + \gamma_4 * B + \gamma_5 * D + \gamma_6 * F$$



#### Variables de Decisión

- $T_i$ como el Torque aplicado al Molino $i$.
- $I$ como la Imbibición % Fibra aplicada al tándem de Molinos.

#### Función Objetivo

$$ E = \alpha_1 * \sum_{i=1}^{6} T_i + \alpha_2 * I + \alpha_3 * F + \alpha_4 * C$$

Y el problema de optimización se plantea por medio de una función de la siguiente forma:
$$ \max E = \max \{ \alpha_1 * \sum_{i=1}^{6} T_i + \alpha_2 * I + \alpha_3 * F + \alpha_4 * C \}$$

Sujeta a:

- $T_i \leq T_{i-max}, \forall i \in \{1,2,3,4,5,6\}$ (El torque en el Molino $i$ no supere el torque máximo permitido en el Reductor $i$).

- $P_i = T_i * \omega_i = T_i * \frac{2 * \pi}{60} *\frac{S_i}{R_i} \leq P_{i-max}, \forall i \in \{1,2,3,4,5,6\}$ (La potencia en el Molino $i$ no supere la potencia máxima permitida en el motor eléctrico $i$).

- $H(T_1,T_6,I,B,D,F) \leq H_{max}$ (La humedad del bagazo no supere la humedad máxima permitida).

- $W(T_1,T_2,T_3,T_4,T_5,T_6,F) \leq W_{max}$ (Un ratio de molienda específico).

- $T_i \geq T_{i-min},  \forall i \in \{1,2,3,4,5,6\}$ (El torque en el Molino $i$ no sea inferior al torque mínimo permitido en el Reductor $i$).

### Modelo de Optimización

In [3]:
# crear modelo de optimización
w = Model('Molinos_TB')

Set parameter Username
Set parameter LicenseID to value 2610087


### Variables de Decisión

In [4]:
# variables de decision
T1 = w.addVar(name='Torque_M1')
T2 = w.addVar(name='Torque_M2')
T3 = w.addVar(name='Torque_M3')
T4 = w.addVar(name='Torque_M4')
T5 = w.addVar(name='Torque_M5')
T6 = w.addVar(name='Torque_M6')
TT = w.addVar(name='Torque_Total')
Imb = w.addVar(name='Imbibicion')

In [5]:
# Mediciones de Proceso

# Potencias Motores Eléctricos (kW)
JT55N101_pv = 917
JT55N201_pv = 865
JT55N301_pv = 882
JT55N401_pv = 656
JT55N501_pv = 468
JT55N601_pv = 530

# Velocidad Motores Eléctricos (rpms)
ST55N101_pv = 699
ST55N201_pv = 638
ST55N301_pv = 780
ST55N401_pv = 792
ST55N501_pv = 771
ST55N601_pv = 927

# Variables de Calidad de Caña
Fibra_Caña = 13
Core_Sampler = 128.2
Dia_Zafra = 180
Bagazo_Caña = 26.2

# Parámetros de Proceso

# Torques Máximos Permitidos (kNm)
TQ55N101_max = 2300
TQ55N201_max = 2300
TQ55N301_max = 2300
TQ55N401_max = 2300
TQ55N501_max = 2300
TQ55N601_max = 2300

# Torques Mínimos Permitidos (kNm)
TQ55N101_min = 300
TQ55N201_min = 300
TQ55N301_min = 300
TQ55N401_min = 300
TQ55N501_min = 300
TQ55N601_min = 300

# Potencias Máximas Motores Eléctricos (kW)
JT55N101_max = 900
JT55N201_max = 900
JT55N301_max = 650
JT55N401_max = 650
JT55N501_max = 650
JT55N601_max = 800


In [6]:
# función objetivo
# E = 9.58167929e+01 + 1.30564865e-11*TT + 2.17531435e-03*Imb - 2.10645544e-01*Fibra_Caña + 2.90500395e-02*Core_Sampler
# Debido a la escala pequeña del coeficiente del Torque Total, se multiplica la función objetivo por 1e+11 que hace más estable numéricamente la optimización. Este escalamiento no cambia el problema.

w.setObjective(9.58167929e+12 + 1.30564865e+00*TT + 2.17531435e+08*Imb - 2.10645544e+10*Fibra_Caña + 2.90500395e+09*Core_Sampler,GRB.MAXIMIZE)


In [7]:
# restricciones
w.addConstr(5.28199559e+02 + 1.88219396e-01*T1 - 3.71569031e-09*T2 + 5.51366311e-02*T3 + 6.46980777e-02*T4 + 6.04199874e-02*T5 + 6.54760007e-03*T6 - 3.91315483e+01*Fibra_Caña <= 650,'ratio_molienda_max')
w.addConstr(4.73046858e+01 + 1.15348642e-04*T1 - 2.48697368e-04*T6 + 7.95713706e-04*Imb + 1.92101683e+00*Bagazo_Caña - 8.77759126e-05*Dia_Zafra - 3.80080926e+00*Fibra_Caña <= 49,'humedad')

# Torque Total
w.addConstr(TT == (T1 + T2 + T3 + T4 + T5 + T6))

# Torques máximos permitidos (reductores)
w.addConstr(T1  <= TQ55N101_max,'torque_max_m1')
w.addConstr(T2  <= TQ55N201_max,'torque_max_m2')
w.addConstr(T3  <= TQ55N301_max,'torque_max_m3')
w.addConstr(T4  <= TQ55N401_max,'torque_max_m4')
w.addConstr(T5  <= TQ55N501_max,'torque_max_m5')
w.addConstr(T6  <= TQ55N601_max,'torque_max_m6')

# Torques máximos permitidos (motores eléctricos)
w.addConstr(T1 <= (60*JT55N101_max)/(2*np.pi*ST55N101_pv/180),'torque_max_m1_electrico')
w.addConstr(T2 <= (60*JT55N201_max)/(2*np.pi*ST55N201_pv/180),'torque_max_m2_electrico')
w.addConstr(T3 <= (60*JT55N301_max)/(2*np.pi*ST55N301_pv/130),'torque_max_m3_electrico')
w.addConstr(T4 <= (60*JT55N401_max)/(2*np.pi*ST55N401_pv/130),'torque_max_m4_electrico')
w.addConstr(T5 <= (60*JT55N501_max)/(2*np.pi*ST55N501_pv/130),'torque_max_m5_electrico')
w.addConstr(T6 <= (60*JT55N601_max)/(2*np.pi*ST55N601_pv/180),'torque_max_m6_electrico')


# Torques mínimos permitidos (extracción)
w.addConstr(T1  >= TQ55N101_min,'torque_min_m1')
w.addConstr(T2  >= TQ55N201_min,'torque_min_m2')
w.addConstr(T3  >= TQ55N301_min,'torque_min_m3')
w.addConstr(T4  >= TQ55N401_min,'torque_min_m4')
w.addConstr(T5  >= TQ55N501_min,'torque_min_m5')
w.addConstr(T6  >= TQ55N601_min,'torque_min_m6')

# Agua de Imbibición % Fibra
w.addConstr(Imb  <= 250,'max_imbibicion')

<gurobi.Constr *Awaiting Model Update*>

In [8]:
# guardar modelo para inspección
w.write('WEC.lp')
w.display()

Maximize
  9680261589190.0 + 1.30564865 Torque_Total + 217531435.0 Imbibicion
Subject To
ratio_molienda_max: 0.188219396 Torque_M1 + -3.71569031e-09 Torque_M2 + 0.0551366311
Torque_M3 + 0.0646980777 Torque_M4 + 0.0604199874 Torque_M5 + 0.00654760007 Torque_M6 <=
 630.511
humedad: 0.000115348642 Torque_M1 + -0.000248697368 Torque_M6 + 0.000795713706
 Imbibicion <= 0.790993
R2: -1.0 Torque_M1 + -1.0 Torque_M2 + -1.0 Torque_M3 + -1.0 Torque_M4 + -1.0 Torque_M5
 + -1.0 Torque_M6 + Torque_Total = 0
  torque_max_m1: Torque_M1 <= 2300
  torque_max_m2: Torque_M2 <= 2300
  torque_max_m3: Torque_M3 <= 2300
  torque_max_m4: Torque_M4 <= 2300
  torque_max_m5: Torque_M5 <= 2300
  torque_max_m6: Torque_M6 <= 2300
  torque_max_m1_electrico: Torque_M1 <= 2213.14
  torque_max_m2_electrico: Torque_M2 <= 2424.74
  torque_max_m3_electrico: Torque_M3 <= 1034.51
  torque_max_m4_electrico: Torque_M4 <= 1018.83
  torque_max_m5_electrico: Torque_M5 <= 1046.58
  torque_max_m6_electrico: Torque_M6 <= 1483.39
  t

/var/folders/r8/vjfpl_c11yj96w72yy_xrcj80000gn/T/ipykernel_2653/3541112219.py:3: DeprecationWarning: Model.display() is deprecated
  w.display()


In [9]:
# motor de optimización
w.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.4.0 24E248)

CPU model: Apple M2 Max
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 22 rows, 8 columns and 35 nonzeros
Model fingerprint: 0x40fc95bd
Coefficient statistics:
  Matrix range     [4e-09, 1e+00]
  Objective range  [1e+00, 2e+08]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e-01, 2e+03]
Presolve removed 22 rows and 8 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.7346445e+12   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  9.734644460e+12


In [10]:
# variables optimizadas

humedad = 4.73046858e+01 + 1.15348642e-04*T1.x - 2.48697368e-04*T6.x + 7.95713706e-04*Imb.x + 1.92101683e+00*Bagazo_Caña - 8.77759126e-05*Dia_Zafra - 3.80080926e+00*Fibra_Caña
ratio = 5.28199559e+02 + 1.88219396e-01*T1.x - 3.71569031e-09*T2.x + 5.51366311e-02*T3.x + 6.46980777e-02*T4.x + 6.04199874e-02*T5.x + 6.54760007e-03*T6.x - 3.91315483e+01*Fibra_Caña

print('--- Indicadores de Extracción ----')
print('Ratio de Molienda Proyectado: %s t/h'%np.round(ratio,2))
print('Humedad de Bagazo Proyectada: %s %%'%np.round(humedad,2))
print('Extracción de Sacarosa Proyectada: %s %%'%np.round(w.objVal/1e+11,2))
print('Agua Imbibición (%% Fibra): %s %%'%np.round(Imb.x,2))
print('Torque Total Molinos: %s kNm'%np.round(TT.x,2))
print('---------- Molino No. 1 ----------')
print('Torque: %s kNm --> Potencia Media: %s kW, Velocidad: %s rpms' % (np.round(T1.x,3),JT55N101_pv,np.round(180*(60*JT55N101_pv)/(2*np.pi*T1.x),2)))
print('---------- Molino No. 2 ----------')
print('Torque: %s kNm --> Potencia Media: %s kW, Velocidad: %s rpms' % (np.round(T2.x,3),JT55N201_pv,np.round(180*(60*JT55N201_pv)/(2*np.pi*T2.x),2)))
print('---------- Molino No. 3 ----------')
print('Torque: %s kNm --> Potencia Media: %s kW, Velocidad: %s rpms' % (np.round(T3.x,3),JT55N301_pv,np.round(180*(60*JT55N301_pv)/(2*np.pi*T3.x),2)))
print('---------- Molino No. 4 ----------')
print('Torque: %s kNm --> Potencia Media: %s kW, Velocidad: %s rpms' % (np.round(T4.x,3),JT55N401_pv,np.round(180*(60*JT55N401_pv)/(2*np.pi*T4.x),2)))
print('---------- Molino No. 5 ----------')
print('Torque: %s kNm --> Potencia Media: %s kW, Velocidad: %s rpms' % (np.round(T5.x,3),JT55N501_pv,np.round(180*(60*JT55N501_pv)/(2*np.pi*T5.x),2)))
print('---------- Molino No. 6 ----------')
print('Torque: %s kNm --> Potencia Media: %s kW, Velocidad: %s rpms' % (np.round(T6.x,3),JT55N601_pv,np.round(180*(60*JT55N601_pv)/(2*np.pi*T6.x),2)))

--- Indicadores de Extracción ----
Ratio de Molienda Proyectado: 631.95 t/h
Humedad de Bagazo Proyectada: 48.29 %
Extracción de Sacarosa Proyectada: 97.35 %
Agua Imbibición (% Fibra): 250.0 %
Torque Total Molinos: 9096.45 kNm
---------- Molino No. 1 ----------
Torque: 2213.142 kNm --> Potencia Media: 917 kW, Velocidad: 712.2 rpms
---------- Molino No. 2 ----------
Torque: 2300.0 kNm --> Potencia Media: 865 kW, Velocidad: 646.45 rpms
---------- Molino No. 3 ----------
Torque: 1034.507 kNm --> Potencia Media: 882 kW, Velocidad: 1465.48 rpms
---------- Molino No. 4 ----------
Torque: 1018.833 kNm --> Potencia Media: 656 kW, Velocidad: 1106.74 rpms
---------- Molino No. 5 ----------
Torque: 1046.583 kNm --> Potencia Media: 468 kW, Velocidad: 768.63 rpms
---------- Molino No. 6 ----------
Torque: 1483.386 kNm --> Potencia Media: 530 kW, Velocidad: 614.14 rpms
